## Keras General Adversarial Networks (GANS)
This tutorial is on how to create a General Adversarial Network Based on a auto encoder to generate simulated data. This pariticular work as adapted from:

https://github.com/eriklindernoren/Keras-GAN#adversarial-autoencoder

This essentially builds a very basic GAN using only feedforward neural networks, and no complex layers


#### Components of a Gan

###### Generator
+ takes random data as input
+ predicts output that looks like our training data
+ trainable weights
+ Not Training Seperately, training inside stacked model

###### Discriminator
+ takes real and fake data combined in one array
+ trains against [0,1] fake vs real labels

###### Stacked Generator and Discriminator
+ Generator + Discriminator (with the weights set to be no trainable ie frozen)

##### Create Data Function
+ takes in real data, generator
+ returns real_data, fake_data of the same size and shape

##### Training Function
+ takes in real_data, discriminator, stacked model
+ During Each Epoc 
    + trains the distriminator on a 50/50 mix of real and fake images against [0,1] labels
    + trains stacked model with disciminator wieghts frozed on input noise, against an array of all 1s\
        + this force the generator part of the stacked model to improve it's generation ablity


In [2]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, merge
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.models import load_model
from keras.optimizers import Adam

from keras import losses
from keras.utils import to_categorical
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
import pandas as pd
import os
import logging
# base logger setup, to standardize logging across classes
try:
    logger.debug('testing logger')
except:
    name = 'GANs'
    formatter = logging.Formatter(fmt='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    logger.addHandler(handler)

C:\Python36\Lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


##### Data 
For this experiment, data from sklearn's Boston Housing is used

In [3]:
# loads data 
X_train = load_boston()['data']
feature_names = load_boston()['feature_names']
# normalizes the data
scaler = StandardScaler().fit(X_train )
X_train = scaler.transform(X_train)


#### Generator Setup
Generator is a feed forward neural network with one hidden layer.  The output layer has the same number of nodes as the real data has columns.  Keep in mind this will not be trained on it's own, but will be stacked with the discriminator

+ n_real_inputs : int number of columns on the input data
+ n_hidden_nodes : int number of nodes in the generator's hidden layer
+ n_fake_inputs : int size of the array of random data used as input to the generator

Array of Noise In -> Array of Fake Data Out

In [4]:
n_real_inputs = 13
n_hidden_nodes = 5
n_fake_inputs = 10

# create a function that returns agenerator
def create_generator(n_real_inputs, n_hidden_nodes, n_fake_inputs):
    inputs = Input(shape=[n_fake_inputs])
    dense_layer0 = Dense(n_hidden_nodes, kernel_initializer='glorot_normal')(inputs)
    batch_norm_layer0 = BatchNormalization()(dense_layer0)
    activation_layer0 = Activation('relu')(batch_norm_layer0)
    dense_layer1 =  Dense(n_real_inputs, kernel_initializer='glorot_normal')(activation_layer0) 
    outputs = Activation('sigmoid')(dense_layer1)
    generator = Model(inputs, outputs)
    generator.compile(loss='categorical_crossentropy', optimizer='adam')
    return generator

generator = create_generator(n_real_inputs = 13, n_hidden_nodes = 5, n_fake_inputs = 10)
generator.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
batch_normalization_1 (Batch (None, 5)                 20        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 13)                78        
_________________________________________________________________
activation_2 (Activation)    (None, 13)                0         
Total params: 153
Trainable params: 143
Non-trainable params: 10
_________________________________________________________________


In [5]:
# rns a test of the generator
test = np.random.normal(size=(2, 10))
generator.predict(test)

array([[0.48387054, 0.51035076, 0.44514462, 0.48686144, 0.48740128,
        0.6040142 , 0.44040295, 0.5304882 , 0.5187814 , 0.48172677,
        0.59508747, 0.6323703 , 0.5526812 ],
       [0.4469352 , 0.48444942, 0.3489605 , 0.48228285, 0.5327607 ,
        0.82372767, 0.25842804, 0.583084  , 0.5310224 , 0.50137466,
        0.7568576 , 0.85362273, 0.6785073 ]], dtype=float32)

#### Discriminator Setup
The discriminator is a feedforeward nerual network with one hidden layer and drop out

+ n_real_inputs : int number of columns on the input data
+ n_hidden_nodes : int number of nodes in the generator's hidden layer
+ dropout_rate float (probablity at which nodes of the hidden layer randomly are set to zero)

Array of Real and Fake Data,  Arrray [0,1] labels in -> Probality of real or fake out



In [6]:
def create_discriminator(n_real_inputs, n_hidden_nodes, dropout_rate):
    inputs = Input(shape=[n_real_inputs])
    dense_layer0 = Dense(n_hidden_nodes, kernel_initializer='glorot_normal')(inputs)
    activation_layer0 = Activation('relu')(dense_layer0)
    drop0 = Dropout(dropout_rate)(activation_layer0)
    dense_layer1 =  Dense(2, kernel_initializer='glorot_normal')(drop0) 
    outputs = Activation('softmax')(dense_layer1)
    discriminator  = Model(inputs, outputs)
    discriminator.compile(loss='sparse_categorical_crossentropy', optimizer='adam',  metrics=['accuracy'])
    return discriminator
discriminator = create_discriminator(n_real_inputs, n_hidden_nodes=5,dropout_rate=.1 )
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 13)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 70        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 12        
_________________________________________________________________
activation_4 (Activation)    (None, 2)                 0         
Total params: 82
Trainable params: 82
Non-trainable params: 0
_________________________________________________________________


#### Model Stacking 
generator and discriminator are stacked to create one model
+ In the model the Discriminator Model is frozen so that error can backpropigate to the generator
+ The array of noise is paried with an array of [1]s.  Since the discrinator is learning sort out fakes, will force the generator in improve it's ability to make them

Array of Noise, Arrray [1] labels in -> Conversion to Fake Data -> Probality of real or fake out

In [7]:
# Create a stacked version of the model with the generator wieghts frozen
def create_stacked_model(generator, discriminator):
    stacked_model = Sequential()
    generator.trainable = False
    stacked_model.add(generator)
    stacked_model.add(discriminator)
    stacked_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
    return stacked_model

stacked_model = create_stacked_model(generator, discriminator)
stacked_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 13)                153       
_________________________________________________________________
model_2 (Model)              (None, 2)                 82        
Total params: 235
Trainable params: 82
Non-trainable params: 153
_________________________________________________________________


#### Create Training Data Function
This function takes real data and a generator, and returns array of real data and an array of fake data.  The real data is randomly select to get the batch size correcly.  Each batch is half real and half fake data

+ X_train: real data input
+ generator Keras Model
+ batch: int size of the arrays return (num_rows)


real_data, generator in -> real_data, fake_data

In [8]:
# define the training functions
def create_training_data(X_train, generator, batch=10):
    n_obs, n_inputs = X_train.shape
    half_batch = int(batch/2)
    n_fake_inputs = generator.get_input_shape_at(0)[1]
    # randomly sample real data 
    index =  np.random.randint(0, n_obs, half_batch)
    real_data = X_train[index,:]
    # generate fake data 
    noise = np.random.normal(0, 1, (half_batch, n_fake_inputs))
    fake_data =  generator.predict(noise)
    return real_data, fake_data 


#### Primary Training Function
+ X_train: training array
+ generator: Keras Generator Model
+ stacked_model:  Keras Generator stacked with Discriminator (discriminator weights frozen)
+ epochs: int number of batches  
+ batch: int number of rows per batch, (half real and half fake data)

In [9]:
def training(X_train, generator, discriminator, epochs=10, batch=10):
    # create the stacked model
    stacked_model = create_stacked_model(generator, discriminator)
    # get the input data shape
    n_obs, n_inputs = X_train.shape
    half_batch = int(batch/2)
    n_fake_inputs = generator.get_input_shape_at(0)[1]
    for e in range(epochs):
        real_data, fake_data = create_training_data(X_train, generator, batch)
        combined_data = np.concatenate((real_data, fake_data))
        real_labels = np.ones(half_batch)
        fake_labels = np.zeros(half_batch)
        combined_labels = np.append(real_labels, fake_labels)
        
        # discriminator loss 
        disc_loss = discriminator.train_on_batch(combined_data ,combined_labels)
        
        # create noise and labels of 1,
        noise =  np.random.normal(0, 1, (batch, n_fake_inputs))
        y_mislabled =  np.ones((batch))
        
        # genertor loss 
        gen_loss = stacked_model.train_on_batch(x=noise,y=y_mislabled)
        logger.info('epoch:{0}/{1}, disc_loss: {2}, gen_loss:{3}'.format(e, epochs, disc_loss, gen_loss))

    generator = stacked_model.layers[0]
    discriminator = stacked_model.layers[1]
    return generator, discriminator

generator, discriminator  = training(X_train, generator, discriminator,  epochs=10, batch=1000)

2019-10-04 08:43:04,247 - GANs - INFO - epoch:0/10, disc_loss: [0.7689241, 0.659], gen_loss:0.9203584790229797
2019-10-04 08:43:04,284 - GANs - INFO - epoch:1/10, disc_loss: [0.8061289, 0.674], gen_loss:0.9177113175392151
2019-10-04 08:43:04,320 - GANs - INFO - epoch:2/10, disc_loss: [0.8111212, 0.638], gen_loss:0.9144372344017029
2019-10-04 08:43:04,355 - GANs - INFO - epoch:3/10, disc_loss: [0.7906476, 0.656], gen_loss:0.9099361896514893
2019-10-04 08:43:04,390 - GANs - INFO - epoch:4/10, disc_loss: [0.77769876, 0.664], gen_loss:0.9106239676475525
2019-10-04 08:43:04,425 - GANs - INFO - epoch:5/10, disc_loss: [0.7969023, 0.654], gen_loss:0.8984869122505188
2019-10-04 08:43:04,460 - GANs - INFO - epoch:6/10, disc_loss: [0.7496962, 0.663], gen_loss:0.9002196788787842
2019-10-04 08:43:04,495 - GANs - INFO - epoch:7/10, disc_loss: [0.79726297, 0.665], gen_loss:0.8940260410308838
2019-10-04 08:43:04,529 - GANs - INFO - epoch:8/10, disc_loss: [0.7813691, 0.67], gen_loss:0.8821180462837219


#### Generating Fake Data 
using random normals to generate face data, this essentialy reverses the scaler to get the data back on the original scale

In [10]:
## make some fake data
def make_fake_data(batch, generator, scaler):
    n_fake_inputs =  generator.get_weights()[0].shape[0]
    noise =  np.random.normal(0, 1, (batch, n_fake_inputs))
    fake_data = generator.predict(noise)
    fake_data_unscaled = scaler.inverse_transform(fake_data)
    return fake_data_unscaled 
print('FAKE :')
pd.DataFrame(make_fake_data(batch=4, generator=generator, scaler=scaler), columns=feature_names)    


FAKE :


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,9.630709,25.205938,13.705052,0.127833,0.615989,6.729080,86.306015,4.993907,14.583076,503.467377,19.134132,406.529541,17.683327
1,9.482731,25.808048,13.730895,0.138355,0.620942,6.716838,84.361420,4.945702,14.397435,515.505920,19.137049,403.286377,17.494619
2,8.869923,24.657532,13.952588,0.160760,0.621343,6.720174,82.104767,4.911867,14.150250,512.250427,19.320797,406.522400,17.175816
3,9.590015,26.893044,13.610885,0.140755,0.627422,6.718511,82.590912,4.913502,14.294861,530.572998,19.074656,400.094849,17.509432


In [11]:
print('REAL :')
test_data = load_boston()['data'] 
pd.DataFrame(test_data[0:4,:], columns=feature_names)

REAL :


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94


#### Looking for Fake Data or Outliers
The discriminator function can be used to detect outliers or look for fake data 



In [12]:
def is_fake(x, dicriminator, scaler):
    x_scaled = scaler.transform(x)
    prob =  dicriminator.predict(x_scaled)
    return prob
 
pd.DataFrame(is_fake(test_data, discriminator, scaler), columns=['fake', 'real']).head()

,fake,real
0,0.735093,0.264907
1,0.407069,0.592931
2,0.507168,0.492832
3,0.543324,0.456676
4,0.556500,0.443500


#### Saving the Models
If you want to save the model
and load it later for use 

In [13]:
path = "_model.h5"
model = generator
# serialize weights to HDF5
model.save(path,  overwrite=True)
print("Saved model to {}".format(path))

Saved model to _model.h5


#### loading Model

In [14]:
# load model
model_loaded = load_model(path)
# summarize model.
model_loaded.summary()
make_fake_data(2, model_loaded, scaler)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
batch_normalization_1 (Batch (None, 5)                 20        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 13)                78        
_________________________________________________________________
activation_2 (Activation)    (None, 13)                0         
Total params: 153
Trainable params: 143
Non-trainable params: 10
_________________________________________________________________


array([[7.6574674e+00, 3.0479498e+01, 1.2478597e+01, 1.8674630e-01,
        6.1072046e-01, 6.8288379e+00, 7.4392029e+01, 5.0441360e+00,
        1.4764271e+01, 5.2169354e+02, 2.0199808e+01, 4.3279443e+02,
        1.7316753e+01],
       [7.0966034e+00, 2.4860298e+01, 1.4669879e+01, 2.0017436e-01,
        5.9405363e-01, 6.5576000e+00, 8.4546837e+01, 5.0787811e+00,
        1.1993657e+01, 5.2492279e+02, 1.9207066e+01, 4.1454782e+02,
        1.5964885e+01]], dtype=float32)

In [15]:
#cleanup
os.remove(path)
